# Create photoreduction forcing files

In [2]:
import numpy as np
import netCDF4 as nc

%matplotlib inline

Parameterization for photo reduction:
- Higher sol_flux means stronger photo reduction.
- Background reduction value: 1
- Photo-enhanced reduction value maximum: 58

In [3]:
imin, imax = 1479, 2179
jmin, jmax = 159, 799

##### Load files

In [4]:
mask      = nc.Dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mesh1.nc')
tmask     = np.array(mask.variables['tmask'])
land_mask = np.ma.masked_where((tmask[0,:,:,:] > 0.1), tmask[0,:,:,:])
lons      = np.array(mask.variables['nav_lon'])
lats      = np.array(mask.variables['nav_lat'])
tmask     = tmask[0,:,imin:imax,jmin:jmax]

##### Functions

In [5]:
def load_variables(filename):
    # Load solar flux variable from ocean-ice model dynamics fields
    a = nc.Dataset(filename)
    
    sol_flux = np.array(a.variables['iocesflx'][:,0,:,:])     
    
    return sol_flux

In [6]:
def red_rate_calc(year):
    
    # domain dimensions:
    imin, imax = 1479, 2179
    jmin, jmax = 159, 799
    
    # calculate reduction rate based on the solar flux from ANHA12 dynamics fields from Paul Myers' group
    # (http://knossos.eas.ualberta.ca/anha/anhatable.php)
    sol_flux      = load_variables(f'/data/brogalla/ANHA12/icemod_ave/ANHA12_EXH006_{year}_monthly.nc')
    max_sol_flux  = sol_flux[:,imin:imax,jmin:jmax].max()
    mean_sol_flux = sol_flux[:,imin:imax,jmin:jmax].mean()
    
    normalized_solar_flux = np.abs(sol_flux - mean_sol_flux)/max_sol_flux
    red_rate              = normalized_solar_flux * 58
    
    red_rate[red_rate < 1]  = 1
    red_rate[red_rate > 58] = 58
    
    return red_rate

In [86]:
red_rate_new = red_rate_calc(2002)

In [89]:
def save_file(filename, field1):
    ncd = nc.Dataset(filename, 'w', zlib=True)

    ncd.createDimension('x',len(mesh.dimensions['x']))
    ncd.createDimension('y',len(mesh.dimensions['y']))
    ncd.createDimension('time_counter',None)
    
    # variables
    red             = ncd.createVariable('kred', 'float64', ('y','x'))
    red.units       = 'Unitless'
    red.long_name   = 'Photoreduction multiplier'  
    red.coordinates = 'nav_lon nav_lat'
    red[:]          = field1[:]

    ncd.close()
    return

In [95]:
def red_year(year, savefile=False):
    
    red_rateyear = red_rate_calc(year)
    
    if savefile:
        location='/data/brogalla/reduction_forcing/20210817/'
        
        for i in range(1,13):
            save_file(f'{location}red_y{year}m{i:02}.nc',red_rateyear[i-1,:,:])
        
    
    return red_rateyear

##### Calculations

In [96]:
for year in range(2002,2003):
    red_rate = red_year(year, savefile=True)